# Parametrização do modelo

Instalação de biblioteca.

In [1]:
!pip install -q -U google-generativeai

Configurar API key.

In [2]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('key')

genai.configure(api_key=GOOGLE_API_KEY)

Configuração do modelo.

In [3]:
generation_config = {
    'candidate_count': 1, # Apenas uma resposta.
    'temperature': 0.5,    # Temperatura 0 mais criativo, 1 menos criativo.
}

Configuração de segurança.

In [4]:
safety_settings = {
    'HARASSMENT': 'block_most', #block_none(não bloquear nada) block_few(pouco), block_some(algumas), block_most(maioria)
    'HATE': 'block_most',
    'SEXUAL': 'block_most',
    'DANGEROUS': 'block_some',
}

Incializa o modelo generativo.

In [5]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              generation_config=generation_config)
                              #safety_settings=safety_settings)

Gerar o texto.

In [6]:
response = model.generate_content('Era uma vez ...')

Armazena o histórico do chat.

In [7]:
chat = model.start_chat(history=[])

Função para armazenar as opções de escolha do usuário.

In [8]:
alternatives = {}

def generate_alternatives(response_text, model):
  alternatives = {}

  alternatives['A'] = model.generate_content(f'Gere uma frase para continuar esta história (max. 10 palavras): {response.text}').text
  alternatives['B'] = model.generate_content(f'Gere uma frase para continuar esta história (max. 10 palavras): {response.text}').text
  alternatives['C'] = model.generate_content(f'Gere uma frase para continuar esta história (max. 10 palavras): {response.text}').text
  alternatives['D'] = model.generate_content(f'Gere uma frase para continuar esta história (max. 10 palavras): {response.text}').text

  print('')
  for option, text in alternatives.items():
      print(f"{option} - {text}")
  print('')

  return alternatives

Entrada do usuário.

In [ ]:
class Response:
    def __init__(self, text):
        self.text = text

response = Response('Era uma vez ...')
prompt = ''

while prompt != "FIM":
  response = model.generate_content(f'Continue a escrever a história: {response.text}')

  print(response.text)

  # Função para armazenar as opções de escolha do usuário.
  generate_alternatives(response.text, model)

  prompt = input("Rumo: ").upper()

  response = chat.send_message(prompt)
  #prompt = input("Rumo: ").upper()

  if len(prompt) == 1:
    if alternatives is not None:
      try:
        response.text = f'{response.text} - {alternatives[prompt]}'
      except KeyError:
        print("Opção inválida.")

  else:
     # A entrada tem mais de um caractere.
     # Quando o usuário deseja inserir outros elementos na história
     response.text = f'{response.text} - {prompt}'

